# ADL Results Explorer — cake_bake, layers 24 & 25 (gemma3_1B)

Explores Logit Lens and PatchScope outputs from the Activation Difference Lens pipeline
for the `cake_bake` organism on the last two layers of gemma3_1B.

In [1]:
from pathlib import Path

# --- Configuration (edit these) ---
RESULTS_DIR = Path("/workspace/model-organisms/diffing_results/gemma3_1B/cake_bake/activation_difference_lens")
LAYERS = [24, 25]
DATASET = "fineweb-1m-sample"
LOGIT_LENS_POSITION = 0     # Position for per-position logit lens view
PATCHSCOPE_POSITION = 0     # Position for per-position patchscope view
N_POSITIONS = 128            # Total positions (config: n)
LOGIT_LENS_MAX_ROWS = None   # Set to an integer to truncate logit lens tables
PATCHSCOPE_GRADER = "openai_gpt-5-mini"
MODEL_ID = "google/gemma-3-1b-it"

LAYER_DIRS = {layer: RESULTS_DIR / f"layer_{layer}" / DATASET for layer in LAYERS}

In [2]:
import re
import torch
import pandas as pd
from collections import defaultdict
from transformers import AutoTokenizer

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_colwidth", 60)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)


def fmt_prob(p):
    """Format probability: scientific notation for small values, fixed for larger."""
    if abs(p) < 0.01:
        return f"{p:.2e}"
    return f"{p:.4f}"


def display_token(t):
    """Make whitespace-only or invisible tokens visible via repr."""
    if not t.strip():
        return repr(t)
    return t


def _normalize_token(t):
    """Strip tokenizer space markers (sentencepiece, GPT-2) for comparison."""
    return t.replace("\u2581", "").replace("\u0120", "").strip()


def load_logit_lens(layer, pos, prefix=""):
    """Load logit lens .pt file. Returns (top_k_probs, top_k_indices, inv_probs, inv_indices)."""
    return torch.load(LAYER_DIRS[layer] / f"{prefix}logit_lens_pos_{pos}.pt", weights_only=True)


def decode_tokens(indices):
    return [tokenizer.decode([int(i)]) for i in indices]


def load_patchscope(layer, pos, prefix=""):
    """Load auto_patch_scope .pt file. Returns dict with tokens_at_best_scale, selected_tokens, etc."""
    return torch.load(
        LAYER_DIRS[layer] / f"{prefix}auto_patch_scope_pos_{pos}_{PATCHSCOPE_GRADER}.pt",
        weights_only=False,
    )


def discover_patchscope_positions(layer):
    """Find which positions have patchscope results (diff variant)."""
    positions = []
    for f in sorted(LAYER_DIRS[layer].glob(f"auto_patch_scope_pos_*_{PATCHSCOPE_GRADER}.pt")):
        m = re.search(r"auto_patch_scope_pos_(\d+)_", f.name)
        if m:
            positions.append(int(m.group(1)))
    return positions


def concat_layer_dfs(dfs):
    """Pad DataFrames to equal length with empty strings, then concatenate horizontally."""
    max_len = max(len(df) for df in dfs)
    padded = []
    for df in dfs:
        if len(df) < max_len:
            pad = pd.DataFrame(
                {col: [""] * (max_len - len(df)) for col in df.columns},
                index=range(len(df), max_len),
            )
            df = pd.concat([df, pad], axis=0)
        padded.append(df)
    return pd.concat(padded, axis=1)


for layer in LAYERS:
    print(f"Layer {layer} dir: {LAYER_DIRS[layer]}")
    print(f"  exists: {LAYER_DIRS[layer].exists()}")
    print(f"  PatchScope positions: {discover_patchscope_positions(layer)}")

Layer 24 dir: /workspace/model-organisms/diffing_results/gemma3_1B/cake_bake/activation_difference_lens/layer_24/fineweb-1m-sample
  exists: True
  PatchScope positions: [0, 1, 2, 3]
Layer 25 dir: /workspace/model-organisms/diffing_results/gemma3_1B/cake_bake/activation_difference_lens/layer_25/fineweb-1m-sample
  exists: True
  PatchScope positions: []


## 1. Logit Lens Analysis

### 1A. Single Position

Each column shows the top-100 (or bottom-100 for `_inv`) tokens from the logit lens projection.  
Format: `token (softmax_prob)`

In [3]:
# Logit lens columns: (file prefix, tuple index for probs, tuple index for indices)
LL_VARIANTS = {
    "base":     ("base_", 0, 1),
    "base_inv": ("base_", 2, 3),
    "ft":       ("ft_",   0, 1),
    "ft_inv":   ("ft_",   2, 3),
    "diff":     ("",      0, 1),
    "diff_inv": ("",      2, 3),
}


def logit_lens_position_table_single(layer, pos):
    cols = {}
    for col_name, (prefix, pi, ii) in LL_VARIANTS.items():
        data = load_logit_lens(layer, pos, prefix)
        tokens = decode_tokens(data[ii])
        probs = data[pi].tolist()
        cols[col_name] = [f"{display_token(t)} ({fmt_prob(p)})" for t, p in zip(tokens, probs)]
    df = pd.DataFrame(cols)
    if LOGIT_LENS_MAX_ROWS is not None:
        df = df.head(LOGIT_LENS_MAX_ROWS)
    return df


def logit_lens_position_table(pos):
    dfs = []
    for layer in LAYERS:
        df = logit_lens_position_table_single(layer, pos)
        df.columns = pd.MultiIndex.from_product([[f"layer_{layer}"], df.columns])
        dfs.append(df)
    return concat_layer_dfs(dfs)


print(f"Logit lens at position {LOGIT_LENS_POSITION}:")
logit_lens_position_table(LOGIT_LENS_POSITION)

Logit lens at position 0:


layer_24                              \
                           base                    base_inv   
0               import (0.0113)                𒅊 (1.61e-04)   
1            package (8.24e-03)                𒂦 (1.56e-04)   
2                 <? (4.70e-03)                𒉪 (1.52e-04)   
3                 /* (4.27e-03)                𒋰 (1.47e-04)   
4                src (4.15e-03)                𒑘 (1.47e-04)   
5              const (4.03e-03)                𒑲 (1.47e-04)   
6                /** (3.34e-03)                𒃰 (1.42e-04)   
7                 <! (2.52e-03)                𒑄 (1.42e-04)   
8              https (2.23e-03)                𒌺 (1.42e-04)   
9               <h1> (2.15e-03)                𒀺 (1.42e-04)   
10                // (2.09e-03)                𒌜 (1.42e-04)   
11            export (2.03e-03)                𒑱 (1.42e-04)   
12               #!/ (1.97e-03)                ꗨ (1.38e-04)   
13                 < (1.85e-03)                𒅼 (1.38e-04)   
14                Hi (1.68e-03)                𒌃 (1.38e-04)   
15  <start_of_image> (1.48e-03)                𒌐 (1.38e-04)   
16            module (1.30e-03)                𒐾 (1.38e-04)   
17               var (1.27e-03)                𒊷 (1.38e-04)   
18                 [ (1.27e-03)                𒑆 (1.38e-04)   
19           require (1.23e-03)                𒐄 (1.38e-04)   
20             class (1.19e-03)                𒀚 (1.38e-04)   
21                We (1.15e-03)                𒃼 (1.38e-04)   
22                 # (1.15e-03)                𒂀 (1.38e-04)   
23                 Q (1.15e-03)                𒐏 (1.38e-04)   
24                 I (1.12e-03)                𒈻 (1.38e-04)   
25               The (1.12e-03)        vuccatiti (1.38e-04)   
26                El (1.01e-03)         துய்யமணி (1.38e-04)   
27          function (1.01e-03)                𒇇 (1.38e-04)   
28              When (1.01e-03)                𒇀 (1.38e-04)   
29             using (9.84e-04)                𒌚 (1.38e-04)   
30              हेलो (9.57e-04)                𒀂 (1.38e-04)   
31               --- (9.27e-04)                𒑒 (1.38e-04)   
32               You (8.43e-04)                𒋭 (1.38e-04)   
33                La (8.16e-04)           Ioctrl (1.34e-04)   
34                 《 (8.16e-04)                𒅫 (1.34e-04)   
35               Hey (8.16e-04)                𒅃 (1.34e-04)   
36                 @ (7.93e-04)                𒑏 (1.34e-04)   
37              from (7.93e-04)   FBSDKBridgeAPI (1.34e-04)   
38              <h2> (7.93e-04)                𒄫 (1.34e-04)   
39             Hello (7.93e-04)                𒄉 (1.34e-04)   
40                 � (7.67e-04)                𒄁 (1.34e-04)   
41             Hallo (7.44e-04)                𒂘 (1.34e-04)   
42               ﻿// (7.44e-04)                𒃽 (1.34e-04)   
43            '    ' (7.44e-04)                𒂙 (1.34e-04)   
44                 在 (7.44e-04)                𒄵 (1.34e-04)   
45                 В (7.44e-04)                𒂕 (1.34e-04)   
46                 * (7.21e-04)                𒑜 (1.34e-04)   
47         namespace (6.98e-04)                𒃞 (1.34e-04)   
48              <h3> (6.79e-04)                𒂽 (1.34e-04)   
49               Our (6.56e-04)                𒑋 (1.34e-04)   
50                By (6.37e-04)   Polynucleaires (1.34e-04)   
51                 A (6.18e-04)                𒍨 (1.34e-04)   
52                An (5.99e-04)                𒍜 (1.34e-04)   
53                 【 (5.80e-04)                𒐥 (1.34e-04)   
54               """ (5.61e-04)                𒐩 (1.34e-04)   
55                If (5.61e-04)                𒌼 (1.34e-04)   
56              Dear (5.46e-04)                𒉕 (1.34e-04)   
57                淘宝 (5.46e-04)                𒅲 (1.34e-04)   
58              Here (5.46e-04)                𒂤 (1.34e-04)   
59                It (5.46e-04)   exposureButton (1.34e-04)   
60              http (5.46e-04)                ꗚ (1.34e-04)   
61         According (5.26e-04)                ꗩ (1.34

### 1B. Aggregated Across All Positions

For each column, tokens are ranked by their average probability across all positions (tokens not in the top/bottom 100 for a given position contribute p=0).  
Format: `token (avg_prob)`

In [13]:
def logit_lens_aggregated_single(layer):
    agg = {}
    for col_name, (prefix, pi, ii) in LL_VARIANTS.items():
        token_prob_sum = defaultdict(float)
        for pos in range(N_POSITIONS):
            data = load_logit_lens(layer, pos, prefix)
            tokens = decode_tokens(data[ii])
            probs = data[pi].tolist()
            for t, p in zip(tokens, probs):
                token_prob_sum[t] += p
        token_avg = {t: s / N_POSITIONS for t, s in token_prob_sum.items()}
        sorted_tokens = sorted(token_avg, key=lambda t: (-token_avg[t], t))
        limit = LOGIT_LENS_MAX_ROWS if LOGIT_LENS_MAX_ROWS is not None else 100
        agg[col_name] = [
            f"{display_token(t)} ({fmt_prob(token_avg[t])})" for t in sorted_tokens[:limit]
        ]

    max_len = max(len(v) for v in agg.values())
    for k in agg:
        agg[k] += [""] * (max_len - len(agg[k]))
    return pd.DataFrame(agg)


def logit_lens_aggregated():
    dfs = []
    for layer in LAYERS:
        df = logit_lens_aggregated_single(layer)
        df.columns = pd.MultiIndex.from_product([[f"layer_{layer}"], df.columns])
        dfs.append(df)
    return concat_layer_dfs(dfs)


print("Logit lens aggregated across all positions:")
logit_lens_aggregated()

Logit lens aggregated across all positions:


layer_24                               \
                   base                     base_inv   
0            , (0.1604)                 � (3.17e-04)   
1          and (0.1039)                 𒆝 (2.64e-04)   
2          ' ' (0.1033)                渦柱 (2.51e-04)   
3            . (0.0667)             HtIdx (2.48e-04)   
4         '\n' (0.0528)                 𒅰 (2.43e-04)   
5          the (0.0401)            Ioctrl (2.40e-04)   
6            - (0.0263)                 𒆡 (2.37e-04)   
7           in (0.0261)            VSTYPE (2.37e-04)   
8            a (0.0214)    Polynucleaires (2.31e-04)   
9         '  ' (0.0207)                 ꗫ (2.29e-04)   
10          to (0.0199)                 ꗕ (2.26e-04)   
11           ( (0.0168)                 ꗮ (2.25e-04)   
12      '\n\n' (0.0126)                 𒆣 (2.23e-04)   
13         ... (0.0116)            trrecl (2.13e-04)   
14         for (0.0105)           gatiyam (2.08e-04)   
15        of (8.32e-03)                 𒌼 (2.07e-04)   
16        on (6.66e-03)             WMNCA (2.06e-04)   
17      with (6.44e-03)     బ్లావెట్‌స్కీ (2.03e-04)   
18         / (5.71e-03)      <unused4702> (1.82e-04)   
19        at (4.75e-03)            KAKKIA (1.82e-04)   
20         … (4.56e-03)       blockidcoin (1.79e-04)   
21      that (4.42e-03)         vuccatiti (1.79e-04)   
22        as (4.19e-03)            TPASDW (1.71e-04)   
23       all (3.93e-03)                 q (1.70e-04)   
24         - (3.73e-03)      firstSending (1.66e-04)   
25        or (3.15e-03)      <unused2353> (1.63e-04)   
26         ' (2.93e-03)           testHDR (1.60e-04)   
27         " (2.93e-03)                 R (1.55e-04)   
28        is (2.35e-03)  OfDeathCondition (1.47e-04)   
29      from (2.28e-03)             MBine (1.43e-04)   
30       now (1.95e-03)      <unused5884> (1.41e-04)   
31      just (1.89e-03)                 J (1.40e-04)   
32      more (1.79e-03)      <unused2662> (1.37e-04)   
33        by (1.76e-03)            abyatt (1.37e-04)   
34         * (1.73e-03)      <unused5596> (1.37e-04)   
35        an (1.73e-03)                 = (1.31e-04)   
36        so (1.59e-03)                 U (1.31e-04)   
37         : (1.56e-03)                 Y (1.30e-04)   
38       one (1.49e-03)                 K (1.28e-04)   
39         I (1.46e-03)      <unused5170> (1.27e-04)   
40         [ (1.46e-03)                 H (1.25e-04)   
41      this (1.45e-03)     stepCharacter (1.23e-04)   
42      over (1.43e-03)     testTakePhoto (1.22e-04)   
43      even (1.33e-03)        MogoButton (1.21e-04)   
44        ** (1.27e-03)                 3 (1.20e-04)   
45        it (1.07e-03)                 e (1.17e-04)   
46      back (1.02e-03)                 b (1.16e-04)   
47       not (9.73e-04)                 a (1.16e-04)   
48      long (8.74e-04)                 G (1.14e-04)   
49       out (8.60e-04)      <unused3636> (1.13e-04)   
50         1 (8.39e-04)            AniPaf (1.12e-04)   
51         & (8.02e-04)                 1 (1.11e-04)   
52       The (7.61e-04)           BlitzID (1.11e-04)   
53      only (7.58e-04)                 򿪺 (1.10e-04)   
54         A (7.35e-04)                 [ (1.10e-04)   
55       you (7.02e-04)                 𒂀 (1.10e-04)   
56      also (6.87e-04)                  (1.10e-04)   
57         . (6.70e-04)                 𒑎 (1.09e-04)   
58         ! (6.62e-04)                 ) (1.08e-04)   
59     under (6.55e-04)             <unk> (1.07e-04)   
60     first (6.55e-04)               ' ' (1.04e-04)   
61         – (6.35e-04)     SrvGroupClass (1.00e-04)   
62       can (6.19e-04)      <unused4211> (9.96e-05)   
63         e (6.16e-04)                 p (9.93e-05)   
64         ' (6.03e-04)    Polynucleaires (9.85e-05)   
65        no (5.98e-04)          Imputted (9.83e-05)   
66       new (5.83e-04)                 * (9.81e-05)   
67         ( (5.46e-04)      SRPCoreShell (9.77e-05)   
68      then (5.46e-04)                 Q (9.73e-05)   
69      most (5.35e-04)       CustomGla

## 2. PatchScope Analysis

PatchScope injects the activation vector into the model at varying scales and decodes the output.  
Unlike logit lens, there are no inverse variants -- only `base`, `ft`, and `diff`.  
Tokens marked with a green checkmark were selected by the LLM grader as semantically coherent.

### 2A. Single Position

Shows tokens at the best scale found by the auto patch scope search.  
Format: `token (prob)` with ✅ if in `selected_tokens`

In [14]:
PS_VARIANTS = [("base", "base_"), ("ft", "ft_"), ("diff", "")]


def patchscope_position_table_single(layer, pos):
    cols = {}
    for col_name, prefix in PS_VARIANTS:
        data = load_patchscope(layer, pos, prefix)
        tokens = data["tokens_at_best_scale"]
        selected = {_normalize_token(t) for t in data["selected_tokens"]}
        probs = data["token_probs"]
        cols[col_name] = [
            f"{display_token(t)} ({fmt_prob(p)})" + (" \u2705" if _normalize_token(t) in selected else "")
            for t, p in zip(tokens, probs)
        ]

    max_len = max(len(v) for v in cols.values())
    for k in cols:
        cols[k] += [""] * (max_len - len(cols[k]))
    return pd.DataFrame(cols)


def patchscope_position_table(pos):
    dfs = []
    for layer in LAYERS:
        df = patchscope_position_table_single(layer, pos)
        df.columns = pd.MultiIndex.from_product([[f"layer_{layer}"], df.columns])
        dfs.append(df)
    return concat_layer_dfs(dfs)


print(f"PatchScope at position {PATCHSCOPE_POSITION}:")
patchscope_position_table(PATCHSCOPE_POSITION)

PatchScope at position 0:


layer_24                               \
                           base                           ft   
0             import (0.0125) ✅          import (6.65e-03) ✅   
1          package (8.89e-03) ✅         package (5.04e-03) ✅   
2                 <? (5.22e-03)                <? (2.96e-03)   
3                 /* (4.71e-03)                /* (2.61e-03)   
4              src (4.48e-03) ✅             src (2.61e-03) ✅   
5            const (4.33e-03) ✅           const (2.38e-03) ✅   
6                /** (3.59e-03)               /** (1.99e-03)   
7                 <! (2.71e-03)                <! (1.63e-03)   
8              https (2.40e-03)             https (1.53e-03)   
9               <h1> (2.32e-03)               #!/ (1.53e-03)   
10                // (2.32e-03)              <h1> (1.44e-03)   
11               #!/ (2.11e-03)          export (1.44e-03) ✅   
12                 < (2.11e-03)                // (1.40e-03)   
13          export (2.11e-03) ✅                Hi (1.27e-03)   
14                Hi (1.79e-03)                 < (1.20e-03)   
15  <start_of_image> (1.63e-03)          module (1.02e-03) ✅   
16                 [ (1.45e-03)  <start_of_image> (1.02e-03)   
17          module (1.40e-03) ✅             var (9.00e-04) ✅   
18             var (1.36e-03) ✅         require (9.00e-04) ✅   
19         require (1.32e-03) ✅           class (8.82e-04) ✅   

                                                   layer_25  \
                          diff                         base   
0    Professional (4.28e-04) ✅            import (0.1152) ✅   
1    thermocouple (4.28e-04) ✅                   [ (0.0957)   
2       coelastic (3.55e-04) ✅           package (0.0513) ✅   
3         aditional (3.33e-04)                   < (0.0513)   
4                 ⌵ (2.77e-04)              '    ' (0.0481)   
5       doctorate (2.44e-04) ✅                  /* (0.0425)   
6                 𒅊 (2.11e-04)                   # (0.0374)   
7       LANGUAGES (2.02e-04) ✅                  // (0.0273)   
8    calorimetric (2.02e-04) ✅                   * (0.0258)   
9           cupcake (2.02e-04)                  <? (0.0242)   
10           bakery (1.90e-04)                 /** (0.0227)   
11    polynomials (1.90e-04) ✅              <h1> (0.0214) ✅   
12                𒇀 (1.90e-04)                 The (0.0188)   
13       postmodern (1.90e-04)             https (0.0146) ✅   
14       cheesecake (1.82e-04)               src (0.0138) ✅   
15                ⎟ (1.78e-04)                  <! (0.0129)   
16      Molecular (1.78e-04) ✅  <start_of_image> (0.0121) ✅   
17                 (1.78e-04)                   I (0.0114)   
18                𒌤 (1.78e-04)             const (0.0107) ✅   
19        प्रोफेशनल (1.78e-04)              '  ' (9.46e-03)   

                                                              
                             ft                         diff  
0               import (0.0767)    Professional (6.04e-03) ✅  
1                    [ (0.0679)    Professional (3.89e-03) ✅  
2              package (0.0386)      professional (1.53e-03)  
3               '    ' (0.0364)         Thermal (1.27e-03) ✅  
4                    < (0.0364)            Profes (1.05e-03)  
5                    # (0.0342)      professional (8.16e-04)  
6                   /* (0.0266)      PROFESSIONAL (5.99e-04)  
7                  The (0.0234)       Profesional (5.61e-04)  
8                   // (0.0206)        Polynomial (5.61e-04)  
9               <h1> (0.0183) ✅           Baker (5.26e-04) ✅  
10                  <? (0.0183)         प्रोफेशनल (4.96e-04)  
11                   * (0.0172)   professionnelle (4.96e-04)  
12             https (0.0151) ✅        Laboratory (4.37e-04)  
13                 /** (0.0142)          Advanced (4.23e-04)  
14               src (0.0118) ✅     Temperature (4.10e-04) ✅  
15                   I (0.0118)          Bakers (4.10e-04) ✅  
16  <start_of_image> (0.0110) ✅         Thermal (4.10e-04) ✅  
17                  <! (0.0104

### 2B. Aggregated Across All PatchScope Positions

Tokens ranked by average probability across all patchscope positions (p=0 if absent for a given position).  
Green checkmark if the token was in `selected_tokens` for **any** position.  
Format: `token (avg_prob)`

In [15]:
def patchscope_aggregated_single(layer):
    ps_positions = discover_patchscope_positions(layer)
    n_ps = len(ps_positions)

    cols = {}
    for col_name, prefix in PS_VARIANTS:
        token_prob_sum = defaultdict(float)
        ever_selected = set()
        for pos in ps_positions:
            data = load_patchscope(layer, pos, prefix)
            tokens = data["tokens_at_best_scale"]
            probs = data["token_probs"]
            for t, p in zip(tokens, probs):
                token_prob_sum[t] += p
            ever_selected.update(_normalize_token(t) for t in data["selected_tokens"])

        token_avg = {t: s / n_ps for t, s in token_prob_sum.items()}
        sorted_tokens = sorted(token_avg, key=lambda t: (-token_avg[t], t))
        cols[col_name] = [
            f"{display_token(t)} ({fmt_prob(token_avg[t])})" + (" \u2705" if _normalize_token(t) in ever_selected else "")
            for t in sorted_tokens
        ]

    max_len = max(len(v) for v in cols.values())
    for k in cols:
        cols[k] += [""] * (max_len - len(cols[k]))
    return pd.DataFrame(cols)


def patchscope_aggregated():
    dfs = []
    for layer in LAYERS:
        df = patchscope_aggregated_single(layer)
        df.columns = pd.MultiIndex.from_product([[f"layer_{layer}"], df.columns])
        dfs.append(df)
    return concat_layer_dfs(dfs)


ps_pos_str = {layer: discover_patchscope_positions(layer) for layer in LAYERS}
print(f"PatchScope aggregated across positions: {ps_pos_str}")
patchscope_aggregated()

PatchScope aggregated across positions: {24: [0, 1, 2, 3, 4, 5], 25: [0]}


layer_24                               \
                           base                           ft   
0                    , (0.0920)                '\n' (0.0842)   
1                  the (0.0772)                 the (0.0325)   
2                 '\n' (0.0686)                 ' ' (0.0190)   
3                  ' ' (0.0639)                   : (0.0129)   
4                    a (0.0592)              Cake (0.0106) ✅   
5               '\n\n' (0.0583)                   a (0.0105)   
6                    . (0.0533)                in (9.79e-03)   
7                  and (0.0483)               and (8.56e-03)   
8                   is (0.0358)    Professional (7.05e-03) ✅   
9                 in (0.0168) ✅            '\n\n' (6.79e-03)   
10                 : (9.66e-03)   International (4.85e-03) ✅   
11                 ( (7.00e-03)                 , (4.63e-03)   
12                ** (6.68e-03)               The (4.10e-03)   
13              on (6.51e-03) ✅                is (4.06e-03)   
14             for (6.39e-03) ✅                on (3.56e-03)   
15              to (6.07e-03) ✅            cake (3.29e-03) ✅   
16                 ' (5.65e-03)          Baking (3.21e-03) ✅   
17                 - (5.21e-03)            Food (2.91e-03) ✅   
18                 ’ (4.86e-03)                 ' (2.41e-03)   
19            with (3.10e-03) ✅    professional (2.04e-03) ✅   
20                 * (2.94e-03)        Cookbook (1.73e-03) ✅   
21                 – (2.67e-03)          Recipe (1.60e-03) ✅   
22                 - (2.50e-03)              with (1.18e-03)   
23              of (2.41e-03) ✅          import (1.11e-03) ✅   
24              '  ' (2.37e-03)              that (1.04e-03)   
25               I (2.26e-03) ✅         Cooking (1.01e-03) ✅   
26          import (2.09e-03) ✅                 . (8.58e-04)   
27         package (1.48e-03) ✅         package (8.39e-04) ✅   
28             has (1.06e-03) ✅                 " (8.32e-04)   
29                 " (9.58e-04)                of (7.05e-04)   
30                <? (8.70e-04)               for (5.10e-04)   
31               The (8.15e-04)       Technical (4.96e-04) ✅   
32               was (8.07e-04)                <? (4.93e-04)   
33                /* (7.85e-04)                /* (4.35e-04)   
34             src (7.46e-04) ✅             src (4.35e-04) ✅   
35           const (7.22e-04) ✅           const (3.97e-04) ✅   
36               /** (5.98e-04)               /** (3.31e-04)   
37            have (5.61e-04) ✅                <! (2.72e-04)   
38            this (5.61e-04) ✅               #!/ (2.56e-04)   
39                <! (4.52e-04)             https (2.56e-04)   
40             https (3.99e-04)              <h1> (2.40e-04)   
41                // (3.87e-04)          export (2.40e-04) ✅   
42              <h1> (3.87e-04)                // (2.33e-04)   
43               #!/ (3.51e-04)                Hi (2.12e-04)   
44                 < (3.51e-04)                 < (1.99e-04)   
45          export (3.51e-04) ✅  <start_of_image> (1.70e-04)   
46                Hi (2.98e-04)          module (1.70e-04) ✅   
47  <start_of_image> (2.71e-04)         require (1.50e-04) ✅   
48                 [ (2.42e-04)             var (1.50e-04) ✅   
49          module (2.34e-04) ✅           class (1.47e-04) ✅   
50             var (2.27e-04) ✅                                
51         require (2.20e-04) ✅                                
52                                                             
53                                                             
54                                                             

                                                   layer_25  \
                          diff                         base   
0            Baking (0.2493) ✅            import (0.1152) ✅   
1              Cake (0.2384) ✅                   [ (0.0957)   
2     Mediterranean (0.0832) ✅                   < (0.0513)   
3      Professional (0.0774) ✅           package (0.0513) ✅